# Analysing magnetic materials using STEM-DPC

This notebook shows how to use the `pyXem` library to analyse 4-D scanning transmission electron microscopy (STEM) data, specifical magnetic materials using differential phase contrast (DPC). For more information about this imaging method, see the Wikipedia article on Scanning Transmission Electron Microscopy, which has a subsection on DPC: https://en.wikipedia.org/wiki/Scanning_transmission_electron_microscopy#Differential_phase_contrast

The data we'll be looking at IS STEM data recorded on a Merlin fast pixelated electron detector system, where the objective lens has been turned off.
This allows for magnetic information to be extracted, by carefully mapping the beam shifts.

More documentation about pyXem is found at https://pyxem.org

## Importing libraries

The first step is setting the plotting toolkit

In [ ]:
%matplotlib qt5

Then import the library itself

In [ ]:
import hyperspy.api as hs

In [ ]:
import pyxem as pxm

### Loading data

As the file is pretty big, we load it lazily.

In [ ]:
s = hs.load("magnetic_stem_dpc.zspy", lazy=True)

### Plotting the data

Lets first have a look at the data

In [ ]:
s.plot()

### Navigation in the detector dimensions

Now, move the navigator across the direct beam, and look for the magnetic stripe in the signal plot.

In [ ]:
s_transpose = s.T

In [ ]:
s_transpose.plot()

This visualizes the shift of the beam, which is caused by the beam passing through the ferromagnetic domains in the material.

However, it is not very quantitative. So lets try to extract the beam shifts using center of mass.

### Extracting the beam shift

There are many ways of extracting the beam shift. One way is using center of mass, which is a fairly simple method.

In [ ]:
s_com = s.get_direct_beam_position(method="center_of_mass")

This returns a `BeamShift` class, which will be explored more later. What we need to know is that is it basically a HyperSpy `Signal1D` class, where the x-beam shifts are in the first signal index (`s.isig[0]`), while the y-shifts are in the second signal index (`s.isig[1]`).

To plot this, we first need to compute it.

In [ ]:
s_com.compute()

In [ ]:
s_com.plot()

### Correcting d-scan

With the beam shift extracted, we will remove the effects of impure beam shift (d-scan).
This is due to various instrument misalignments, and leads to a change in beam position in the probe plane becoming a shift of the beam in the detector plane.
Luckily, in most situations, the d-scan is linear across the dataset, meaning it can be removed using a simple plane subtraction.

However, for the full version of this dataset (if you didn't crop it earlier), a simple plane subtraction is not enough to remove all the effects of this d-scan.
See `fpd.ransac_tools.ransac_im_fit` for more advanced ways of removing it, for example in https://zenodo.org/record/3466591/files/d001_get_dpc_raw_signal.py

In [ ]:
s_linear_plane = s_com.get_linear_plane()

In [ ]:
s_linear_plane.plot()

To subtract the fitted plane from the `s_com` signal, we simply subtract it.

In [ ]:
s_com_corr = s_com - s_linear_plane

This gives us a corrected version of the 

In [ ]:
s_com_corr.plot()

### Visualization

Now we can visualize the signal as a magnitude and direction maps: `get_magnitude_phase_signal`, `get_magnitude_signal` and `get_color_image_with_indicator`.

The two former returns a HyperSpy signal, while the latter interfaces directly with the matplotlib backend making it more customizable.

In [ ]:
s_color = s_com_corr.get_magnitude_phase_signal()
s_color.plot()

`get_magnitude_signal` gives the magnitude of the beam shift vector. Which can be useful for visualizing the domain walls.

In [ ]:
s_magnitude = s_com_corr.get_magnitude_signal()
s_magnitude.plot()

The `get_color_image_with_indicator` method has a large degree of customizability, which is useful when making images for presentations, posters or articles.

By default it returns a matplotlib figure object, which can be saved directly

In [ ]:
pxm.utils.plotting.plot_beam_shift_color(s_com_corr)

Making this into a figure which we can save to a file

In [ ]:
fig = pxm.utils.plotting.plot_beam_shift_color(s_com_corr)
fig.savefig("dpc_image.png")

## Bivariate histogram

Lastly, we can visualize the 2-dimensional histogram of the beam shifts.

In [ ]:
s_hist = s_com_corr.get_bivariate_histogram()
s_hist.plot(cmap='viridis')

Grabbing just the largest stripe

In [ ]:
s_com_corr.plot()

In [ ]:
s_com_corr_crop = s_com_corr.inav[21.66:27.8, 5.5:11.9]

In [ ]:
s_com_corr_crop.plot()

In [ ]:
s_com_corr_crop.get_bivariate_histogram(bins=50).plot(cmap="viridis")